# <center>Phân tích ma trận trong Python - Đồ án 2</center>

## Mục lục
* [Thực hành](#c1)
    * [Trị riêng - Vector riêng](#c11)
    * [Chéo hóa](#c12)
    * [Chéo hóa trực giao](#c13)
* [Đồ án 2: Image Processing](#c2)
    * [Nội dung đồ án](#c21)
    * [Quy định nộp bài](#c22)
    * [Quy định chấm bài](#c23)

## Thực hành <a class="anchor" id="c1"></a>

Trong lab này, chúng ta sẽ tìm hiểu về phân tích ma trận sử dụng `NumPy`.

Cho ma trận:
$$A = \begin{bmatrix}
    3 & -2 & 0\\ 
    -2 & 3 & 0\\ 
    0 & 0 & 5
    \end{bmatrix}$$

In [91]:
import numpy as np

In [92]:
A = np.array([[3, -2, 0],
              [-2, 3, 0],
              [0, 0, 5]])

In [93]:
def is_close(A, B, eps=10e-3):
    return np.all(np.abs(A - B) < eps)

Các phân tích được giới thiệu trong lab này là:
1. Tìm trị riêng và vector riêng
2. Chéo hóa
3. Chéo hóa trực giao

### Trị riêng - Vector riêng <a class="anchor" id="c11"></a>

#### Sử dụng thuật toán [Power iteration](https://en.wikipedia.org/wiki/Power_iteration) và Hotelling's Deflation [[1](https://web.stanford.edu/~lmackey/papers/deflation-nips08.pdf), [2](https://www.robots.ox.ac.uk/~sjrob/Teaching/EngComp/ecl4.pdf)]

In [94]:
def eigen_power_iteration(A, n_iter=1000, eps=10e-5):
    ''' 
    Estimate the dominant eigenvalue and eigenvector of a square matrix using the power iteration method

    Parameters
    ----------
    A : numpy.ndarray
        The square matrix for which to find the dominant eigenvalue and eigenvector
    n_iter : int, default=1000
        The maximum number of iterations to perform
    eps : float, default=10e-5
        The tolerance for convergence. The iteration stops when the change in the eigenvector falls below this value

    Returns
    -------
    tuple (float, numpy.ndarray)
        A tuple containing the estimated dominant eigenvalue (lamb) and the corresponding eigenvector (b_k).
    '''

    # Generate a random vector b_k and init a zero vector b_k_pre
    b_k = np.random.rand(A.shape[1])
    b_k_pre = np.zeros(A.shape[1])

    # Iterate for a maximum of n_iter steps to find the dominant eigenvector
    for _ in range(int(n_iter)):
        # Calculate the dot product of the matrix A and the current eigenvector estimate b_k
        numerator = np.dot(A, b_k)
        denominator = np.linalg.norm(numerator)
        
        b_k = numerator / denominator
        
        # Break the loop when the change in the eigenvector falls below the tolerance (eps)
        if np.all(np.abs(b_k - b_k_pre) < eps):
            break
        
        b_k_pre = b_k
        
    # Estimate the dominant eigenvalue using the Rayleigh quotient
    lamb = ((b_k @ A) @ b_k) / (b_k @ b_k)

    return lamb, b_k


def my_eigens(A, n_iter=1000, eps=10e-5):
    ''' 
    Find all eigenvalues and eigenvectors of a square matrix using the power iteration method with Hotelling's deflation

    Parameters
    ----------
    A : numpy.ndarray
        The square matrix for which to find all eigenvalues and eigenvectors
    n_iter : int, default=1000
        The maximum number of iterations to perform for each eigenvalue estimation using the power iteration method
    eps : float, default=10e-5
        The tolerance for convergence in the power iteration method

    Returns
    -------
    tuple (numpy.ndarray, numpy.ndarray)
        A tuple containing two NumPy arrays:
        - eigenvalues (shape (n,)): An array containing all the computed eigenvalues of the matrix A
        - eigenvectors (shape (n, n)): A 2D array where each column represents an eigenvector, corresponding to the eigenvalue in the same position in the eigenvalues array
    '''

    eigenvalues = []
    eigenvectors = []
    
    n_rows = A.shape[0]
    
    # Find all possible eigenvalues and eigenvectors
    for _ in range(n_rows):
        lamb, b_k = eigen_power_iteration(A, n_iter, eps)
        
        eigenvalues.append(lamb)
        eigenvectors.append(b_k)
        
        # Apply Hotelling's deflation to remove the influence of the already computed eigenvalue-eigenvector pair from A
        b_k = b_k.reshape(-1, 1) # Reshape as a column vector
        A = A - lamb * (b_k @ b_k.T)
        
    return np.array(eigenvalues), np.array(eigenvectors).T

In [95]:
my_eigenvalues, my_eigenvectors = my_eigens(A)

my_eigenvalues, my_eigenvectors

(array([5.        , 5.        , 1.00000002]),
 array([[ 3.26340890e-01, -6.27280908e-01,  7.07178499e-01],
        [-3.26324288e-01,  6.27321885e-01,  7.07035046e-01],
        [ 8.87138142e-01,  4.61503972e-01, -1.18932930e-04]]))

In [96]:
# Evaluate the first pair of eigenvalue and eigenvector
my_lamb = my_eigenvalues[0]
my_X = my_eigenvectors[:, 0].reshape(-1, 1)

A @ my_X, my_lamb * my_X

(array([[ 1.63167125],
        [-1.63165464],
        [ 4.43569071]]),
 array([[ 1.63170445],
        [-1.63162144],
        [ 4.43569071]]))

In [97]:
is_close(A @ my_X, my_lamb * my_X)

True

#### Thư viện `np.linalg`

In [98]:
np_eigenvalues, np_eigenvectors = np.linalg.eig(A)

np_eigenvalues, np_eigenvectors

(array([5., 1., 5.]),
 array([[ 0.70710678,  0.70710678,  0.        ],
        [-0.70710678,  0.70710678,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]))

In [99]:
# Evaluate the first pari of eigenvalue and eigenvector
np_lamb = np_eigenvalues[0]
np_X = np_eigenvectors[:, 0].reshape(-1, 1)

A @ np_X, np_lamb * np_X

(array([[ 3.53553391],
        [-3.53553391],
        [ 0.        ]]),
 array([[ 3.53553391],
        [-3.53553391],
        [ 0.        ]]))

In [100]:
is_close(A @ np_X, np_lamb * np_X)

True

### Chéo hóa <a class="anchor" id="c12"></a>

In [101]:
def my_diag(A):
    ''' 
    Computes the diagonal matrix (D), the eigenvector matrix (P), and its inverse (P_inv) from a square matrix using its eigenvalues and eigenvectors

    Parameters
    ----------
    A : numpy.ndarray
        The square matrix for which to compute the diagonal matrix, eigenvector matrix, and its inverse

    Returns
    -------
    tuple (numpy.ndarray, numpy.ndarray, numpy.ndarray)
        A tuple containing three NumPy arrays:
        - P_matrix (shape (n, n)): The eigenvector matrix, where each column represents an eigenvector of the input matrix A
        - D_matrix (shape (n, n)): The diagonal matrix containing the eigenvalues of A on the diagonal
        - P_inv_matrix (shape (n, n)): The inverse of the eigenvector matrix

    '''

    # Find eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Rearrange the eigenvectors so they correspond to the sorted eigenvalues in descending order
    sorted_idx = np.argsort(eigenvalues)[::-1]  # Quick question: What does this code mean?
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    # Form the result matrices
    P_matrix = eigenvectors
    P_inv_matrix = np.linalg.inv(P_matrix)
    D_matrix = np.diag(eigenvalues)
    
    return P_matrix, D_matrix, P_inv_matrix

In [102]:
P, D, P_inv = my_diag(A)
(P, D, P_inv)

(array([[ 0.        ,  0.70710678,  0.70710678],
        [ 0.        , -0.70710678,  0.70710678],
        [ 1.        ,  0.        ,  0.        ]]),
 array([[5., 0., 0.],
        [0., 5., 0.],
        [0., 0., 1.]]),
 array([[ 0.        ,  0.        ,  1.        ],
        [ 0.70710678, -0.70710678,  0.        ],
        [ 0.70710678,  0.70710678,  0.        ]]))

In [103]:
D, P_inv @ A @ P

(array([[5., 0., 0.],
        [0., 5., 0.],
        [0., 0., 1.]]),
 array([[ 5.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  5.00000000e+00, -2.63285810e-16],
        [ 0.00000000e+00, -6.36938863e-16,  1.00000000e+00]]))

In [104]:
is_close(D, P_inv @ A @ P)

True

### Chéo hóa trực giao <a class="anchor" id="c13"></a>

In [105]:
def my_orth_diag(A):
    ''' 
    Computes the diagonal matrix (D), orthogonal matrix (Q), and its transpose (Q.T) from a square matrix using its eigenvalues and eigenvectors

    Parameters
    ----------
    A : numpy.ndarray
        The square matrix for which to compute the diagonal matrix, orthogonal matrix, and its transpose

    Returns
    -------
    tuple (numpy.ndarray, numpy.ndarray, numpy.ndarray)
        A tuple containing three NumPy arrays:
        - Q_matrix (shape (n, n)): The orthogonal matrix obtained from the eigenvectors of A using QR decomposition
        - D_matrix (shape (n, n)): The diagonal matrix containing the eigenvalues of A on the diagonal
        - Q_transpose (shape (n, n)): The transpose of the orthogonal matrix Q_matrix
    '''

    # Find eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Rearrange the eigenvectors so they correspond to the sorted eigenvalues in descending order
    sorted_idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_idx]
    eigenvectors = eigenvectors[:, sorted_idx]
    
    # Obtain the orthogonal matrix from eigenvectors using QR decomposition
    Q_matrix, _ = np.linalg.qr(eigenvectors)
    
    # Form other result matrices
    D_matrix = np.diag(eigenvalues)
    Q_transpose = Q_matrix.T
    
    return Q_matrix, D_matrix, Q_transpose

In [106]:
(Q, D, Q_T) = my_orth_diag(A)
(Q, D, Q_T)

(array([[ 0.        ,  0.70710678, -0.70710678],
        [-0.        , -0.70710678, -0.70710678],
        [-1.        ,  0.        ,  0.        ]]),
 array([[5., 0., 0.],
        [0., 5., 0.],
        [0., 0., 1.]]),
 array([[ 0.        , -0.        , -1.        ],
        [ 0.70710678, -0.70710678,  0.        ],
        [-0.70710678, -0.70710678,  0.        ]]))

In [107]:
D, Q_T @ A @ Q

(array([[5., 0., 0.],
        [0., 5., 0.],
        [0., 0., 1.]]),
 array([[5.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 5.00000000e+00, 2.63285810e-16],
        [0.00000000e+00, 3.13396163e-16, 1.00000000e+00]]))

In [108]:
is_close(D, Q_T @ A @ Q)

True

---

## Đồ án 2: Image Processing <a class="anchor" id="c2"></a>

### Nội dung đồ án <a class="anchor" id="c21"></a>

<ins>Nhắc lại</ins>: Trong đồ án 1, sinh viên đã được giới thiệu về khái niệm lưu trữ ảnh dưới dạng ma trận các điểm ảnh. Mỗi điểm ảnh có thể được biểu diễn bằng một giá trị nguyên (ảnh xám) hoặc một vector (ảnh màu).

Trong đồ án này, sinh viên được yêu cầu thực hiện các chức năng xử lý ảnh cơ bản sau:
    
1. Thay đổi độ sáng cho ảnh (1 điểm)

![img](https://imgur.com/oJ8bTv7.jpg)

2. Thay đổi độ tương phản (1 điểm)

![img](https://imgur.com/wl8MSu3.jpg)

3. Lật ảnh (ngang - dọc) (1 điểm)

![img](https://imgur.com/MOOvIhN.jpg)

4. Chuyển đổi ảnh RGB thành ảnh xám/sepia (2 điểm)

- Ảnh xám

![img](https://imgur.com/XEfRXWE.jpg)

- Ảnh sepia

![img](https://imgur.com/YXUPjHY.jpg)

Keywords: convert RGB to grayscale/sepia

5. Làm mờ/sắc nét ảnh (2 điểm)

- Làm mờ

![img](https://imgur.com/wZT4vUa.jpg)

- Làm sắc nét

![img](https://imgur.com/H2Fq4Ne.jpg)

Tham khảo tại [đây](https://en.wikipedia.org/wiki/Kernel_(image_processing))

6. Cắt ảnh theo kích thước (cắt ở trung tâm) (1 điểm)

![img](https://imgur.com/fXebjfO.jpg)

7. Cắt ảnh theo khung (1 điểm)

- Khung tròn

![img](https://imgur.com/DEpimhC.jpg)

- Khung là 2 hình elip chéo nhau

![img](https://i.imgur.com/fPlYioC.png)

8. Viết hàm main xử lý (1 điểm) với các yêu cầu sau:

- Cho phép người dùng nhập vào tên tập tin ảnh mỗi khi hàm main được thực thi.
- Cho phép người dùng lựa chọn chức năng xử lý ảnh (từ 1 đến 7, đối với chức năng 4 cho phép lựa chọn giữa lật ngang hoặc lật dọc). Lựa chọn 0 cho phép thực hiện tất cả chức năng với tên file đầu ra tương ứng với từng chức năng. Ví dụ:
    - Đầu vào: `cat.png`
    - Chức năng: Làm mờ
    - Đầu ra: `cat_blur.png`

Trong đồ án này, sinh viên <font style='color:red'>**CHỈ ĐƯỢC PHÉP**</font> sử dụng các thư viện sau: `PIL`, `numpy`, `matplotlib`.

Cụ thể, nếu đề yêu cầu sinh viên cài đặt chức năng nào đó, thì sinh viên phải **thực sự cài đặt chức năng** đó mà không phải gọi hàm có sẵn.

- Đối với thư viện PIL và Matplotlib, sinh viên chỉ được sử dụng các hàm sau:
    - `PIL` (`open(), save()` từ `Image`) để đọc và ghi
    - `Matplotlib` (`imshow()` từ `pyplot`) để hiển thị ảnh
    - <ins>Lưu ý:</ins> <font style='color:red'>sử dụng các hàm khác (ngoài các hàm liệt kê trên)</font> $\to$ <font style='color:red'> NHẬN ĐIỂM 0 CHO TOÀN BỘ ĐỒ ÁN</font>

- Được phép sử dụng thư viện `NumPy` tùy ý

<ins>Lưu ý:</ins> Để được **điểm tối đa** cho từng chức năng, thời gian thực thi của mỗi chức năng phải nằm trong khoảng thời gian chấp nhận được. Ví dụ với chức năng làm mờ (phức tạp nhất) có thời gian thực thi trên ảnh với kích thước $512 \times 512$ là dưới 15 giây.

#### Nâng cao - Không bắt buộc (Cộng 0.5 điểm vào điểm đồ án 2)

Sinh viên tìm hiểu và cài đặt chức năng thu nhỏ, phóng to ảnh (zoom out/zoom in)

![img](https://i.imgur.com/DEzRSfe.png)


### Quy định bài nộp <a class="anchor" id="c22"></a>

* Thực hiện toàn bộ bài làm trên 1 tập tin Jupyter Notebook có mẫu theo đúng tập tin `MSSV.ipynb` đính kèm. Mỗi hàm cần có docstrings tương tự như đồ án 1.


* Nộp tập tin `MSSV.zip` được nén từ thư mục MSSV chứa các tập tin sau:
    1. Báo cáo toàn bộ bài làm: `MSSV.pdf`
    2. Mã nguồn: `MSSV.ipynb`


* Trong đó, nội dung tập tin báo cáo gồm có:
    - Thông tin cá nhân: Họ và tên, MSSV
    - Ý tưởng thực hiện, mô tả các hàm chức năng
    - Bảng đánh giá mức độ hoàn thành và hình ảnh kết quả cho từng chức năng (cho một ảnh đầu vào nhất định)

        | **STT** |    **Chức năng/Hàm**    | **Mức độ hoàn thành** | **Ảnh kết quả** |
        |:-------:|:-----------------------:|:---------------------:|:---------------:|
        |    1    | Thay đổi độ sáng        |       [0, 100]%       |                 |
        |    2    | Thay đổi độ tương phản  |       [0, 100]%       |                 |
        |   3.1   | Lật ảnh ngang           |       [0, 100]%       |                 |
        |   3.2   | Lật ảnh dọc             |       [0, 100]%       |                 |
        |   4.1   | RGB thành ảnh xám       |       [0, 100]%       |                 |
        |   4.2   | RGB thành ảnh sepia     |       [0, 100]%       |                 |
        |   5.1   | Làm mờ ảnh              |       [0, 100]%       |                 |
        |   5.2   | Làm sắc nét ảnh         |       [0, 100]%       |                 |
        |    6    | Cắt ảnh theo kích thước |       [0, 100]%       |                 |
        |   7.1   | Cắt ảnh theo khung tròn |       [0, 100]%       |                 |
        |   7.2   | Cắt ảnh theo khung elip |       [0, 100]%       |                 |
        |    8    | Hàm main                |       [0, 100]%       |                 |
        |    9    | Phóng to/Thu nhỏ 2x     |       [0, 100]%       |                 |

    - Báo cáo cần phải căn lề, có số trang, và tài liệu tham khảo
    
    <ins>Lưu ý:</ins> Viết báo cáo sơ sài/thiếu các yêu cầu trên sẽ nhận điểm trừ tương ứng.

* Ví dụ minh họa cây thư mục bài nộp sau khi giải nén tập tin `MSSV.zip` như sau:
    ```
    MSSV
    ├── MSSV.pdf
    └── MSSV.ipynb
    ```

### Quy định chấm bài <a class="anchor" id="c23"></a>

Đây là đồ án chiếm 10%.

Những trường hợp sau đây sẽ bị 0 điểm toàn bộ đồ án:

- Sử dụng các thư viện và các hàm xử lý ảnh có sẵn mà không được cho phép
- Nộp sai quy định
- Không có báo cáo
- Thực thi mã nguồn báo lỗi

<font style="color:red">**LƯU Ý: SAO CHÉP BÀI LÀM CỦA NHAU SẼ BỊ 0 ĐIỂM TOÀN BỘ PHẦN THỰC HÀNH**</font>